### Tensorflow.Keras를 사용한 RNN 모델링.

#### 1. RNN Layer 구조:

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM, Bidirectional, Embedding, TimeDistributed

SimpleRNN layer의 은닉층에서 그대로 출력된다. <br>
파라미터의 수 = Input_dim * Hidden_dim + Hidden_dim * Hidden_dim + Hidden_dim = 10 * 3 + 3 * 3 + 3 = 30 + 9 + 3 = 42. <br>
SimpleRNN layer의 default activation은 tanh() 이다! <br>
return_sequences의 default 값은 False 이다!

In [2]:
my_model = Sequential(name="RNN1")
my_model.add(SimpleRNN(units = 3, input_shape=(2,10)))                # Hidden dimension = 3, sequence length = 2 & input dimension = 10.
#my_model.add(SimpleRNN(units = 3, input_length=2, input_dim=10))     # 위와 같은 의미!
my_model.summary()                                                    # 파라미터의 수는 sequence length는 무관하다!!! 최종 스텝에서 한 번의 출력이 있다.

Model: "RNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


batch의 입력 크기도 명시할 수 있다.

In [3]:
my_model = Sequential(name="RNN2")
my_model.add(SimpleRNN(units=3, batch_input_shape=(8,2,10)))   # batch size = 8, sequence length = 2 & input dimension = 10.
my_model.summary()                      # 파라미터의 수는 batch size나 sequence length와는 무관하다!!!

Model: "RNN2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


return_sequences = True와 같이 설정하면 매 스텝마다 출력이 있다. 

In [4]:
my_model = Sequential(name="RNN3")
my_model.add(SimpleRNN(units=3, batch_input_shape=(8,2,10), return_sequences=True))  # 매 스텝마다 출력이 있다!
my_model.summary()                       # 파라미터의 수는 batch size, sequence length 또는 return sequences 여부와는 무관하다!!!

Model: "RNN3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


이제는 출력층을 하나 추가해 본다.

In [5]:
my_model = Sequential(name="RNN4")
my_model.add(SimpleRNN(units = 3, batch_input_shape=(8,2,10), return_sequences=True)) 
my_model.add(Dense(units = 5))                    # Hidden_dim * Output_dim + Output_dim = 3*5 + 5 = 20 개의 파라미터가 추가적으로 필요하다.
my_model.summary()                                # 파라미터의 수는 batch size, sequence length 또는 return sequences 여부와는 무관하다!!!

Model: "RNN4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (8, 2, 3)                 42        
                                                                 
 dense (Dense)               (8, 2, 5)                 20        
                                                                 
Total params: 62
Trainable params: 62
Non-trainable params: 0
_________________________________________________________________


#### 2. Embedding Layer의 역할과 구조:

In [6]:
# 단순한 RNN.
my_model = Sequential(name="RNN5")
my_model.add(SimpleRNN(units = 30, input_shape=(20,100)))             # Hidden dimension = 30, sequence length = 20 & input dimension = 100.
my_model.summary() # 파라미터의 수 = Input_dim * Hidden_dim + Hidden_dim * Hidden_dim + Hidden_dim = 100 * 30 + 30 * 30 + 30 = 3930

Model: "RNN5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 30)                3930      
                                                                 
Total params: 3,930
Trainable params: 3,930
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Embedding + RNN.
n_input = 1000                    # 단어의 가지수라 해석할 수 있다. 
n_emb = 100                       # n_emb < n_input!
n_seq = None
n_hidden = 30
my_model = Sequential(name="Embedding_RNN")
my_model.add(Embedding(n_input, n_emb, name="Embedding"))
my_model.add(SimpleRNN(units = n_hidden, input_shape=(n_seq,n_emb), name="RNN"))    

In [8]:
# 내부 구조 출력.
# Embedding의 파라미터 수 = n_input * n_emb  = 1000 * 100 = 100000.
# Embedding층의 역할은 차원 축소이다! 
# Embedding층에 정수 인덱스 (< n_index)를 입력하면 차원 축소된 밀집 벡터를 학습하게 된다! ==> 자연어!!!
# RNN의 파라미터 수 = n_emb * n_hidden + n_hidden * n_hidden + n_hidden = 100 * 30 + 30 * 30 + 30 = 3930.
my_model.summary()   

Model: "Embedding_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, None, 100)         100000    
                                                                 
 RNN (SimpleRNN)             (None, 30)                3930      
                                                                 
Total params: 103,930
Trainable params: 103,930
Non-trainable params: 0
_________________________________________________________________


#### 3. Deep RNN 모델:

In [9]:
my_model = Sequential(name="Deep RNN")
my_model.add(SimpleRNN(units = 3, input_shape=(2,10), return_sequences = True))    # return_sequences = True 와 같이 설정되어 있다. 파라미터의 수 = Input_dim * Hidden_dim + Hidden_dim * Hidden_dim + Hidden_dim = 10*3 + 3*3 + 3 = 42.
my_model.add(SimpleRNN(units = 4, return_sequences = True))   # 파라미터의 수 = Hidden_dim1 * Hidden_dim2 + Hidden_dim2 * Hidden_dim2 + Hidden_dim2 =  3*4 + 4*4 + 4 = 32.
my_model.add(SimpleRNN(units = 5, return_sequences = True))   # 파라미터의 수 = Hidden_dim2 * Hidden_dim3 + Hidden_dim3 * Hidden_dim3 + Hidden_dim3 =  4*5 + 5*5 + 5 = 50.           
my_model.summary()    

Model: "Deep RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_5 (SimpleRNN)    (None, 2, 3)              42        
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 2, 4)              32        
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 2, 5)              50        
                                                                 
Total params: 124
Trainable params: 124
Non-trainable params: 0
_________________________________________________________________


#### 4. Bidirectional RNN 모델:

In [10]:
my_model = Sequential(name = "Bi-RNN1")
my_model.add(Bidirectional(SimpleRNN(3), input_shape=(2,10)))  # 파라미터의 수가 정확하게 두배가 된다.
my_model.summary() 

Model: "Bi-RNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 6)                84        
 l)                                                              
                                                                 
Total params: 84
Trainable params: 84
Non-trainable params: 0
_________________________________________________________________


In [11]:
my_model = Sequential(name = "Bi-RNN2")
my_model.add(Bidirectional(SimpleRNN(3), input_shape=(2,10)))  # 파라미터의 수가 정확하게 두배가 된다.
my_model.add(Dense(units = 5))                                 # Output_dim에 해당하는 units만 명시. Bias vector는 단 한개! 3*5 + 3*5 + 5 = 35 개의 파라미터가 추가적으로 필요하다!
my_model.summary()   

Model: "Bi-RNN2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 6)                84        
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 5)                 35        
                                                                 
Total params: 119
Trainable params: 119
Non-trainable params: 0
_________________________________________________________________


In [12]:
my_model = Sequential(name = "Deep Bi-RNN")
my_model.add(Bidirectional(SimpleRNN(3,return_sequences=True), input_shape=(2,10))) # 파라미터의 수가 정확하게 두배가 된다.
my_model.add(Bidirectional(SimpleRNN(3,return_sequences=True)))                     # 파라미터의 수 = Hidden_dim * Hidden_dim * 6 (with 2 cross terms) + Hidden_dim * 2 (2 biases)
my_model.add(Bidirectional(SimpleRNN(3,return_sequences=True)))                     #                  =  3 * 3 * 6 + 3 * 2 = 54 + 6 = 60.
my_model.add(Dense(units = 5))                                                      # 파라미터의 수 = 2*(Hidden_dim * Output_dim) + Output_dim. Bias vector는 단 한개! 
my_model.summary()   

Model: "Deep Bi-RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 2, 6)             84        
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 2, 6)             60        
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 2, 6)             60        
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 2, 5)              35        
                                                                 
Total params: 239
Trainable params: 239
Non-trainable params: 0
_________________________________________________________

#### 5. Numpy로 RNN 코딩하기:

In [13]:
import numpy as np

In [14]:
# 차원 정의.
timesteps = 10     # Sequence length.
input_dim = 4      # 보통은 word vector의 차원.
hidden_dim = 8     # 은닉층의 차원 = 출력의 차원!

In [15]:
# 배열 생성.
inputs = np.random.random((timesteps, input_dim))     # 입력은 2D 배열.
hidden_state_t = np.zeros((hidden_dim,))             # rank 1 배열이며 차원수 = hidden_dim. 0 으로 초기화.

In [16]:
# 가중치 행렬을 정의한다.
Wx = np.random.random((hidden_dim, input_dim))     # (8, 4) : input -> hidden으로 가는 가중치 행렬.
Wh = np.random.random((hidden_dim, hidden_dim))   # (8, 8) : hidden -> hidden으로 가는 가중치 행렬.
b = np.random.random((hidden_dim,))               # (8,)   : bias.

In [17]:
# Time step을 전개한다.
total_hidden_states = []

for input_t in inputs:          # 입력 행렬의 개개 행을 가져온다!!!
    output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b)     #  Ot = tanh(Wx . Xt) + tanh(Wh . Ht-1) + b.
    total_hidden_states.append(list(output_t))                                 # hidden states를 저장해 간다.
    print(np.shape(total_hidden_states))                                       # 계속 커가는 상태를 출력해 준다. 
    hidden_state_t = output_t

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)


In [18]:
# sequence 출력.
total_hidden_states = np.stack(total_hidden_states, axis = 0)                 # Stack the hidden states (which are also outputs).
print(total_hidden_states.round(3))                                           # dim = (timesteps, output_dim).

[[0.982 0.934 0.958 0.953 0.904 0.873 0.932 0.893]
 [1.    1.    1.    1.    1.    1.    1.    1.   ]
 [1.    1.    0.999 1.    1.    1.    1.    1.   ]
 [1.    1.    1.    1.    1.    1.    1.    1.   ]
 [1.    1.    0.999 1.    1.    1.    1.    1.   ]
 [1.    1.    0.998 1.    1.    1.    1.    1.   ]
 [1.    1.    1.    1.    1.    1.    1.    1.   ]
 [1.    1.    0.999 1.    1.    1.    1.    1.   ]
 [1.    1.    1.    1.    1.    1.    1.    1.   ]
 [1.    1.    1.    1.    1.    1.    1.    1.   ]]
